# Data Preparation

In [4]:
import pandas as pd
train_x_df = pd.read_csv('data/train_x_df.csv')
train_y_df = pd.read_csv('data/train_y_df.csv')
test_x_df = pd.read_csv('data/test_x_df.csv')
train_x_df.head()

,sample_id,time,coin_index,open,high,low,close,volume,quote_av,trades,tb_base_av,tb_quote_av
0,0,0,9,0.983614,0.983614,0.983128,0.983246,0.001334,10.650987,0.009855,0.000848,6.771755
1,0,1,9,0.983245,0.983612,0.982453,0.982693,0.001425,11.375689,0.016137,0.000697,5.565188
2,0,2,9,0.982694,0.983612,0.982403,0.983002,0.001542,12.301942,0.014166,0.000905,7.225459
3,0,3,9,0.983009,0.984848,0.983009,0.984486,0.002520,20.134695,0.021557,0.001171,9.353000
4,0,4,9,0.984233,0.984606,0.983612,0.984164,0.002818,22.515448,0.021434,0.001799,14.372534


In [17]:
train_y_df.columns[2:]

Index(['coin_index', 'open', 'high', 'low', 'close', 'volume', 'quote_av',
       'trades', 'tb_base_av', 'tb_quote_av'],
      dtype='object')

In [91]:
import numpy as np
def df2d_to_array3d(df_2d):
    feature_size = len(df_2d.columns[2:]) # 10
    time_size = len(df_2d.time.unique()) # x는 1380, y는 120
    sample_size = len(df_2d.sample_id.unique()) # train은 7661, test는 535
    array_3d = df_2d.iloc[:,2:].values.reshape([sample_size, time_size, feature_size])
    return array_3d

train_x_array = df2d_to_array3d(train_x_df)
train_y_array = df2d_to_array3d(train_y_df)
test_x_array = df2d_to_array3d(test_x_df)

print(f'''
These shape stands for (sample_size, time_step, feature)
train_x_array {train_x_array.shape}
train_y_array {train_y_array.shape}
test_x_array {test_x_array.shape}
''')


These shape stands for (sample_size, time_series, feature)
train_x_array (7661, 1380, 10)
train_y_array (7661, 120, 10)
test_x_array (535, 1380, 10)



# 0번 샘플로만

In [54]:
df_0 = train_x_array[0]
# y = train_y_array
print('(timesteps, features)')
print(df_0.shape)

(timesteps, features)
(1380, 10)


In [123]:
X = list()
y = list()
for i in range(df_0.shape[0]):
    X.append(df_0[i: i + 200])
    y.append(df_0[i+200: i + 320, 4]) # 4 for close
    if i + 320 == df_0.shape[0]:
        break
X = np.array(X)
y = np.array(y)
y = y.reshape(y.shape[0], y.shape[1], 1)
X.shape, y.shape

((1061, 200, 10), (1061, 120, 1))

In [124]:
y[0]

array([[0.98804015],
       [0.98743898],
       [0.98803401],
       [0.98791325],
       [0.98794037],
       [0.98792195],
       [0.98755848],
       [0.98780489],
       [0.98859942],
       [0.98895538],
       [0.98915869],
       [0.99162358],
       [0.98965508],
       [0.99062574],
       [0.98977703],
       [0.99002337],
       [0.99002832],
       [0.99014658],
       [0.98945928],
       [0.98915744],
       [0.98916602],
       [0.99014539],
       [0.99020201],
       [0.99062574],
       [0.99063808],
       [0.99064666],
       [0.98940623],
       [0.98978192],
       [0.99071813],
       [0.99125403],
       [0.99088448],
       [0.99088448],
       [0.9907366 ],
       [0.98747718],
       [0.98842078],
       [0.9890306 ],
       [0.98979062],
       [0.98817444],
       [0.98813748],
       [0.98813623],
       [0.98804384],
       [0.9880414 ],
       [0.98854399],
       [0.98878539],
       [0.98792803],
       [0.98848236],
       [0.98818058],
       [0.988

## 너로 정했다 케라스

케라케라~

In [93]:
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

In [120]:
from keras.layers

ImportError: cannot import name 'LSTM2d' from 'keras.layers' (/Users/duru/opt/anaconda3/lib/python3.8/site-packages/keras/layers/__init__.py)

In [107]:
model = Sequential()
model.add(LSTM(128, activation='relu', return_sequences=True, input_shape=(200, 10)))
model.add(LSTM(128, activation='relu'))
model.add(Dense(120))
model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_10 (LSTM)               (None, 200, 128)          71168     
_________________________________________________________________
lstm_11 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dense_5 (Dense)              (None, 120)               15480     
Total params: 218,232
Trainable params: 218,232
Non-trainable params: 0
_________________________________________________________________


In [121]:
a = X[0]
a = a.reshape(1, a.shape[0], a.shape[1])
model.predict(a)

array([[ 192241.45  ,   -7592.792 ,   20096.314 , -101035.    ,
          77294.92  ,  -51772.96  ,    9258.358 ,  115586.84  ,
          26799.309 ,  209239.1   ,   98617.17  ,  -93243.2   ,
          46724.656 ,  119716.78  , -130397.65  , -158282.39  ,
        -120547.92  ,  135037.89  ,   49139.203 ,  -13115.148 ,
          40239.797 ,  -32138.922 , -157244.69  ,  124285.23  ,
          82971.65  ,   67772.22  ,  105281.586 ,   41613.324 ,
          99747.06  , -187501.81  ,  -29501.709 ,    2183.211 ,
          32345.88  , -146647.05  , -112215.39  , -230457.62  ,
        -204818.94  ,   23084.277 ,   51551.83  ,   34661.938 ,
          16158.457 ,  -54252.254 ,  154453.72  ,  130829.35  ,
         112159.85  , -121985.266 ,   64506.492 ,  -26956.04  ,
        -176460.61  ,   66276.59  , -151646.81  ,    4511.3125,
         125291.3   , -237638.77  ,  -51815.51  , -123181.    ,
         157637.81  , -176819.31  ,   58754.656 ,  205758.52  ,
         -82259.016 ,   93227.47  ,   11

In [99]:
model.fit(X, y)

34/34 [==============================] - 18s 432ms/step - loss: nan
